In [ ]:
!pip install --upgrade usearch simsimd

In [15]:
from usearch.io import load_matrix, save_matrix

In [11]:
vectors = load_matrix(
    'vectors.hbin', 
    # count_rows=int(10_000_000), 
    view=True,
)
vectors.shape

(247154006, 1024)

In [3]:
import numpy as np
from numba import jit, prange

In [4]:
@jit(parallel=True, nopython=False)
def transposed_bit_matrix(vectors):
    # For every dimension we want to extract the column of the matrix
    # quantizing them into bits on the fly. We then have to copy it to
    # guarantee continuity of the memory layout.
    # 
    #   return np.packbits((vectors.T > 0).astype(np.uint8), axis=1).copy()
    #
    # That, however, is extrememly slow, so let's write a NumBa kernel to
    # do this in parallel.
    nvec, ndim = vectors.shape
    # The output array will have the same number of rows but
    # columns need to be divided by 8 since we pack every 8 bits into one byte
    nbits = (nvec + 7) // 8
    packed_bits = np.zeros((ndim, nbits), dtype=np.uint8)
    
    # Moreover, Numba can't deal with `float16`. We want to find positive values.
    # We can represent the values as `uint16` and then check the binary representation
    # against zero and negative masks.
    negative_mask: np.uint16 = 0x8000
    zero_mask: np.uint16 = 0x0000
    vectors = vectors.view(np.uint16)
    
    for i in prange(ndim):
        # The inner loop goes over the columns.
        # We process 8 columns at a time, packing them into a single byte.
        for j in range(0, nvec, 8):
            byte = 0
            # Process each bit; note that we need to handle the case where
            # the number of columns is not a multiple of 8
            for bit in range(8):
                if j + bit < nvec:
                    # Shift the bit into the correct position and add it to the byte
                    scalar = vectors[j + bit, i]
                    scalar_is_positive = (scalar & negative_mask) != 0 and (scalar != zero_mask)
                    byte |= (scalar_is_positive << (7 - bit))
            # Store the packed byte in the corresponding position
            packed_bits[i, j // 8] = byte

    return packed_bits    

In [12]:
bits_fast = transposed_bit_matrix(vectors.view(np.uint16))
bits_fast.shape

(1024, 30894251)

In [6]:
from simsimd import cdist, DistancesTensor

In [13]:
dimensions_similarity: DistancesTensor = cdist(bits_fast, bits_fast, metric="hamming", threads=0) # zero-copy
dimensions_similarity_array: np.ndarray = np.array(dimensions_similarity, copy=True) # now managed by NumPy
dimensions_similarity_array.shape

(1024, 1024)

In [14]:
dimensions_similarity

array([[0.00000000e+00, 7.54424390e+07, 1.32564126e+08, ...,
        1.03616093e+08, 1.20616484e+08, 1.32747058e+08],
       [7.54424390e+07, 0.00000000e+00, 1.48317623e+08, ...,
        5.59479740e+07, 1.22022899e+08, 1.39413497e+08],
       [1.32564126e+08, 1.48317623e+08, 0.00000000e+00, ...,
        1.43602397e+08, 1.24055114e+08, 1.21359838e+08],
       ...,
       [1.03616093e+08, 5.59479740e+07, 1.43602397e+08, ...,
        0.00000000e+00, 1.33712083e+08, 1.56546517e+08],
       [1.20616484e+08, 1.22022899e+08, 1.24055114e+08, ...,
        1.33712083e+08, 0.00000000e+00, 1.35048516e+08],
       [1.32747058e+08, 1.39413497e+08, 1.21359838e+08, ...,
        1.56546517e+08, 1.35048516e+08, 0.00000000e+00]])

In [16]:
dimensions_i32 = dimensions_similarity.astype(np.int32)

In [17]:
dimensions_i32.__array_interface__

{'data': (98440109832176, False),
 'strides': None,
 'descr': [('', '<i4')],
 'typestr': '<i4',
 'shape': (1024, 1024),
 'version': 3}

In [18]:
save_matrix(dimensions_i32, 'dimensions.ibin')

In [19]:
import numpy as np

def select_greedy_rows(distance_matrix, num_rows_to_select):
    # Initialize the set with the index of the first row
    selected_indices = {np.argmax(np.sum(distance_matrix, axis=1))}
    
    # While we need more rows
    while len(selected_indices) < num_rows_to_select:
        max_dist = 0
        max_idx = -1
        
        # Find the row with the largest minimum distance to the selected set
        for i in range(distance_matrix.shape[0]):
            if i not in selected_indices:
                # Minimum distance to any row in the selected set
                min_dist = np.min([distance_matrix[i, j] for j in selected_indices])
                if min_dist > max_dist:
                    max_dist = min_dist
                    max_idx = i
        
        # Add the index with the largest minimum distance
        selected_indices.add(max_idx)
    
    return list(selected_indices)

In [20]:
selected_rows = select_greedy_rows(dimensions_i32, 64)
print(selected_rows)

[640, 257, 6, 135, 774, 395, 908, 140, 655, 785, 148, 916, 661, 404, 794, 158, 416, 162, 35, 292, 805, 679, 169, 938, 560, 434, 51, 692, 315, 700, 573, 829, 703, 959, 961, 193, 835, 331, 75, 973, 338, 979, 471, 218, 604, 352, 226, 482, 868, 995, 870, 872, 250, 366, 751, 752, 750, 371, 884, 755, 115, 631, 378, 509]
